In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
files = [f for f in os.listdir('.') if os.path.isfile(f)]
for f in files:
  print(f)

In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/IR Proj/scraped_CSV.csv')

In [ ]:
df.columns

Index(['Unnamed: 0', 'book_id', 'review_sentences', 'title'], dtype='object')

In [ ]:
def jaccard_similarity(doc1, doc2):
    # print("doc1", doc1, "doc2", doc2)
    union = len(set(doc1 + doc2))
    intersection = len(set(doc1) & set(doc2))

    return intersection/union

In [ ]:
import multiprocessing
import json
import pickle
# df = df.head(4000)
# print(df.shape)

# num_processes = multiprocessing.cpu_count()
num_processes = 100
print(num_processes)

# calculate the chunk size as an integer
chunk_size = int(df.shape[0]/num_processes)

# this solution was reworked from the above link.
# will work even if the length of the dataframe is not evenly divisible by num_processes
# chunks = [df.iloc[df.index[i:i + chunk_size]] for i in range(0, df.shape[0], chunk_size)]
chunks = [df.iloc[i:i + chunk_size,:] for i in range(0, df.shape[0], chunk_size)]

jaccard_matrix = {}
jaccard_rank = {}

print("Loop books")

def func(df):
  for i,book in df.iterrows():
      # print(book.title)
      print(i)
      reviews = book['review_sentences']
      if book.title not in jaccard_matrix:
          jaccard_matrix[book.title] = {}
      if book.title not in jaccard_rank:
          jaccard_rank[book.title] = []

      for j,other in df.iterrows():
          if i == j or other.title in jaccard_matrix[book.title]:
              continue

          rev = other['review_sentences']
          jaccard_score = jaccard_similarity(reviews, rev)
          jaccard_matrix[book.title][other.title] = jaccard_score
          jaccard_rank[book.title].append((other['title'], jaccard_score))
          if other.title not in jaccard_matrix:
              jaccard_matrix[other.title] = {}
          if other.title not in jaccard_rank:
              jaccard_rank[other.title] = []
          jaccard_matrix[other.title][book.title] = jaccard_score
          jaccard_rank[other.title].append((book['title'], jaccard_score))
            
  return df


# create our pool with `num_processes` processes
pool = multiprocessing.Pool(processes=num_processes)

# apply our function to each chunk in the list
result = pool.map(func, chunks)

for i in range(len(result)):
  df.iloc[result[i].index] = result[i]
  # since result[i] is just a dataframe
   # we can reassign the original dataframe based on the index of each chunk

with open('/content/drive/MyDrive/IR Proj/jaccard_matrix.pkl', 'wb') as f:
    pickle.dump(jaccard_matrix, f)

with open('/content/drive/MyDrive/IR Proj/jaccard_rank.pkl', 'wb') as f:
    pickle.dump(jaccard_rank, f)
        
# with open('saved_dictionary.pkl', 'rb') as f:
#     loaded_dict = pickle.load(f)

for i in jaccard_rank:
    sorted(i, key=lambda x: x[1], reverse = True)

with open("/content/drive/MyDrive/IR Proj/jaccard_matrix.json", 'w') as f: 
    json.dump(jaccard_matrix, f)

with open("/content/drive/MyDrive/IR Proj/jaccard_rank.json", 'w') as f: 
    json.dump(jaccard_rank, f)

In [ ]:
eval = [2466, 1257, 5774, 646, 8050, 849, 394, 4174, 203, 139, 4097, 61, 173, 1026, 1422, 1148, 1354, 5082, 343, 184, 366, 1108, 4556, 3438, 363, 604, 1313, 140, 264, 221]


In [ ]:
len(jaccard_rank)

0

In [ ]:
jaccard_matrix

{}